# 🖼️ Week2: GPT-5 이미지 실습 (정답)



> **주의:** Colab 런타임을 재시작하면 `OPENAI_API_KEY` 환경 변수를 다시 설정해야 합니다.



## 진행 순서
1. 환경 준비 및 기본 이미지 생성
2. 이미지 저장 · 표시 및 출력 옵션 실습
3. 멀티턴 이미지 개선(Responses API)
4. 이미지 분석(비전) 기본 실습
5. 선택 과제 & 확장 아이디어



## 1) 환경 준비 및 기본 이미지 생성



In [ ]:
%pip install openai pillow


In [ ]:
import os
import base64
from getpass import getpass

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    OPENAI_API_KEY = getpass("Enter your OPENAI_API_KEY: ")
    os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

print("✅ API 키 설정 완료")



In [ ]:
from openai import OpenAI
from IPython.display import display, Image as DisplayImage

client = OpenAI()
print("✅ 클라이언트 생성 완료")



### GPT Image 실습 정답 코드



In [ ]:
def generate_image(prompt: str, *, size: str = "1024x1024", quality: str = "medium") -> bytes:
    response = client.responses.create(
        model="gpt-5",
        input=prompt,
        tools=[
            {
                "type": "image_generation",
                "size": size,
                "quality": quality,
            }
        ],
    )

    image_data = [
        output.result
        for output in response.output
        if output.type == "image_generation_call"
    ]

    if not image_data:
        raise RuntimeError("이미지 생성 결과를 찾을 수 없습니다.")

    return base64.b64decode(image_data[0])



In [ ]:
# 예시 실행
prompt = "A cozy reading nook with floor-to-ceiling bookshelves and warm lighting"
img_bytes = generate_image(prompt, size="1024x1024", quality="high")
with open("reading_nook.png", "wb") as f:
    f.write(img_bytes)
print("✅ 이미지 생성 및 저장 완료 -> reading_nook.png")



In [ ]:
def show_image(image_path: str) -> None:
    display(DisplayImage(filename=image_path))

show_image("reading_nook.png")



In [ ]:
def refine_image(prompt: str, follow_up: str) -> tuple[str, bytes]:
    initial = client.responses.create(
        model="gpt-5",
        input=prompt,
        tools=[{"type": "image_generation"}],
    )

    first_calls = [
        output
        for output in initial.output
        if output.type == "image_generation_call"
    ]
    if not first_calls:
        raise RuntimeError("초기 이미지가 생성되지 않았습니다.")

    followup = client.responses.create(
        model="gpt-5",
        previous_response_id=initial.id,
        input=follow_up,
        tools=[{"type": "image_generation"}],
    )

    follow_calls = [
        output
        for output in followup.output
        if output.type == "image_generation_call"
    ]
    if not follow_calls:
        raise RuntimeError("후속 이미지가 생성되지 않았습니다.")

    return followup.id, base64.b64decode(follow_calls[0].result)


# 예시 실행 (필요 시 주석 해제)
# base_prompt = "Generate a stylized poster of a futuristic city skyline at sunset"
# follow_up_prompt = "Increase the level of realism and add flying cars"
# refined_id, refined_bytes = refine_image(base_prompt, follow_up_prompt)
# with open("city_refined.png", "wb") as f:
#     f.write(refined_bytes)
# show_image("city_refined.png")


In [ ]:
def describe_image(image_url: str, question: str, *, detail: str = "auto") -> str:
    response = client.responses.create(
        model="gpt-4.1-mini",
        input=[
            {
                "role": "user",
                "content": [
                    {"type": "input_text", "text": question},
                    {
                        "type": "input_image",
                        "image_url": image_url,
                        "detail": detail,
                    },
                ],
            }
        ],
    )
    return response.output_text


# 예시 실행 (필요 시 주석 해제)
# sample_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/1024px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"
# print(describe_image(sample_url, "이미지 속 주요 대상과 분위기를 설명해 줘"))


In [ ]:
def create_file(image_path: str) -> dict:
    with open(image_path, "rb") as f:
        encoded = base64.b64encode(f.read()).decode("utf-8")
    return {
        "type": "input_image",
        "image_base64": encoded,
    }


# 예시 실행 (필요 시 주석 해제)
# local_image_payload = create_file("reading_nook.png")
# vision_response = client.responses.create(
#     model="gpt-4.1-mini",
#     input=[
#         {
#             "role": "user",
#             "content": [
#                 {"type": "input_text", "text": "이 이미지의 분위기를 한 문장으로 요약해 줘"},
#                 local_image_payload,
#             ],
#         }
#     ],
# )
# print(vision_response.output_text)

